# Divergence Vectors on Corridor Experiment

In this notebook, we run various simulations on the corridor experimetn using both HL and ORCA behaviuors.

We use divergence vectors to see if these are able to distinguish between both.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl

import os

from navground import core, sim

import perdiver.perdiver as perdiver
from perdiver.distances import *

plots_dir = "plots"
os.makedirs(plots_dir, exist_ok=True)

We run a few times the corridor experiment.

In [ ]:
length = 8.0
num_steps = 400
width=1.0
num_agents = 31 # 38
num_runs = 30
for behaviour in ["ORCA", "HL"]:
    path=f"experiment_diverg_vect_{behaviour}.h5"
    yaml = f"""
    steps: {num_steps}
    time_step: 0.1
    record_pose: true
    record_twist: true
    runs: {num_runs}
    scenario:
      type: Corridor
      length: {length}
      width: {width} 
      groups:
        -
          type: thymio
          number: {num_agents}
          radius: 0.08
          control_period: 0.1
          speed_tolerance: 0.02
          kinematics:
            type: 2WDiff
            wheel_axis: 0.094
            max_speed: 0.166
          behavior:
            type: {behaviour}
            optimal_speed: 0.12
            horizon: 5.0
            safety_margin: 0.03
          state_estimation:
            type: Bounded
            range: 5.0
    """
    # optimal_speed: 
    #     sampler: normal
    #     mean: 0.2
    #     std_dev: 0.05
    # horizon: 5.0
    # safety_margin: 
    #     sampler: normal
    #     mean: 0.08
    #     std_dev: 0.05
    experiment = sim.load_experiment(yaml)
    experiment.run(keep=False, data_path=path)
    del experiment

We reload both experiment runs and save them into a dictionary of lists.

In [ ]:
runs = {}
# Reload HL simulation
path_HL = "experiment_diverg_vect_HL.h5"
recorded_experiment = sim.RecordedExperiment(path_HL)
runs["HL"] = recorded_experiment.runs
# Reload ORCA simulation
path_ORCA = "experiment_diverg_vect_ORCA.h5"
recorded_experiment = sim.RecordedExperiment(path_ORCA)
runs["ORCA"] = recorded_experiment.runs

Compute the persistence matching diagrams across both simulations, using a step list variable that contains which steps we consider as initial values. In addition, we consider a number of steps shift between timesteps.

In [ ]:
# Set persistence divergence parameters
shift_timesteps = 30
initial_steps = list(range(100, 350, 20))
weight=2
# Compute persistence divergence for all runs
divergence_behaviour = {}
for j, behaviour in enumerate(["ORCA", "HL"]):
    divergence_array_list = []
    for ridx in range(num_runs):
        divergence_list = []
        ps = np.array(runs[behaviour][ridx].poses)
        twists = np.array(runs[behaviour][ridx].twists)
        for idx, start_step in enumerate(initial_steps):
            X = ps[start_step]
            Y = ps[start_step + shift_timesteps]
            vel_X = twists[start_step]
            vel_Y = twists[start_step+shift_timesteps]
            Dist_X, Dist_Y, Dist_Z = perdiver.compute_distance_matrices_timesteps_corridor(X, Y, vel_X, vel_Y, weight, length)
            divergence_vector = np.sort(perdiver.compute_divergence_vector(Dist_X, Dist_Y))
            divergence_list.append(divergence_vector)
        # end for 
        divergence_array_list.append(np.array(divergence_list).transpose())
    # end for
    divergence_behaviour[behaviour] = divergence_array_list
# end for

In [ ]:
divergences = []
for behaviour in divergence_behaviour.keys():
    for divergence_arr in divergence_behaviour[behaviour]:
        divergences.append(divergence_arr.ravel().tolist())

Compute PCA projection to see the distribution of points.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
Y = pca.fit_transform(divergences)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
half_runs = int(num_runs/2)
ax.scatter(Y[:num_runs,0], Y[:num_runs,1], c="black", marker="o", s=20, label="ORCA")
ax.scatter(Y[num_runs:,0], Y[num_runs:,1], c="red", marker="x", s=20, label="HL")
plt.legend()
plt.savefig(plots_dir + "pca_divergence.png")

In [ ]:
for behaviour in divergence_behaviour.keys():
    divergence_arr = divergence_behaviour[behaviour][1]
    ## Save figure 
    fig, ax = plt.subplots(figsize=(10,2))
    mapable = ax.imshow(divergence_arr, aspect="auto", vmax=13, vmin=-13, extent=(initial_steps[0], initial_steps[-1], 0, X.shape[0]))
    ax.set_title(f"Divergence {behaviour}")
    plt.colorbar(mapable)
    plt.tight_layout()
    plt.savefig(f"{plots_dir}/Divergence_vector_evolution_{behaviour}.png")

In [ ]:
from navground.sim.ui import WebUI
from navground.sim.notebook import notebook_view
from navground.sim.replay import RealTimeReplay

web_ui = WebUI(host='127.0.0.1', max_rate=-1)
await web_ui.prepare()

In [ ]:
notebook_view(width=300)

In [ ]:
mixed_ORCA_run = np.argmin(Y[:num_runs,0])
mixed_HL_run = np.argmax(Y[num_runs:,0])

In [ ]:
rt_sim = RealTimeReplay(run=runs["ORCA"][mixed_ORCA_run], factor=20, web_ui=web_ui)
await rt_sim.run()

In [ ]:
rt_sim = RealTimeReplay(run=runs["HL"][mixed_HL_run], factor=20, web_ui=web_ui)
await rt_sim.run()

Now we see what we get by analysing other basic variables, such as variance of angular speed.

In [ ]:
timestep_list = list(range(0, 300, 5))
variables_behaviour = {}
for j, behaviour in enumerate(["ORCA", "HL"]):
    variables_list = []
    for ridx in range(num_runs):
        variables_run = []
        ps = np.array(runs[behaviour][ridx].poses)
        twists = np.array(runs[behaviour][ridx].twists)
        for idx, step in enumerate(timestep_list):
            X = ps[step]
            vel_X = twists[step]
            variables_run.append(np.hstack((np.mean(X, axis=0), np.std(X, axis=0), weight*np.mean(vel_X, axis=0), weight*np.std(vel_X, axis=0))))
        # end for 
        variables_list.append(np.array(variables_run).transpose())
    # end for
    variables_behaviour[behaviour] = np.array(variables_list)

In [ ]:
matrix = np.copy(variables_behaviour["ORCA"])
matrix.shape

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=6, figsize=(10,10))
color_behaviour = {"HL":"red", "ORCA":"black"}
labels = np.array([
    ["x mean", "y mean", "angle mean", "x std", "y std", "angule std"],
    ["v_x mean", "v_y mean", "angle speed mean", "v_x std", "v_y std", "angular speed std"]
]).transpose()
for irow in range(labels.shape[0]):
    for icol in range(labels.shape[1]):
        for behaviour in ["ORCA", "HL"]:
            for variables_run in variables_behaviour[behaviour]:
                ax[irow, icol].set_title(labels[irow, icol])
                ax[irow, icol].plot(list(range(variables_run.shape[1])), variables_run[icol*6 + irow], color=color_behaviour[behaviour])

See how PCA does by using these variables

In [ ]:
runs_coordinates = []
for behaviour in ["HL", "ORCA"]:
    for variables_run in variables_behaviour[behaviour]:
        runs_coordinates.append(variables_run.ravel())

runs_coordinates = np.array(runs_coordinates)

In [ ]:
runs_coordinates.shape

In [ ]:
pca = PCA(n_components=2)
Y_variables = pca.fit_transform(runs_coordinates)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
half_runs = int(num_runs/2)
ax.scatter(Y_variables[:num_runs,0], Y_variables[:num_runs,1], c="black", marker="o", s=20, label="ORCA")
ax.scatter(Y_variables[num_runs:,0], Y_variables[num_runs:,1], c="red", marker="x", s=20, label="HL")
plt.legend()
plt.savefig(plots_dir + "pca_variables.png")

In [ ]:
runs_coordinates.shape

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
num_steps = len(timestep_list)
runs_coordinates_speeds = runs_coordinates[:, 10*int(runs_coordinates.shape[1]/12):11*int(runs_coordinates.shape[1]/12)]
Y_speeds = pca.fit_transform(runs_coordinates_speeds)
half_runs = int(num_runs/2)
ax.scatter(Y_speeds[:num_runs,0], Y_variables[:num_runs,1], c="black", marker="o", s=20, label="ORCA")
ax.scatter(Y_speeds[num_runs:,0], Y_variables[num_runs:,1], c="red", marker="x", s=20, label="HL")
plt.legend()
plt.savefig(plots_dir + "pca_speeds.png")